In [20]:
%%time 
import requests
import csv

def take_1000_posts(domain, offset):
    token = '9ddb8dd19ddb8dd19ddb8dd1389ec0401f99ddb9ddb8dd1fb658ceeec2522bea1361087'
    version = '5.199'
    all_posts = []
    
    session = requests.Session()

    try:
        response = session.get(
            'https://api.vk.com/method/wall.get',
            params={
                'access_token': token,
                'v': version,
                'domain': domain,
                'count': 100,
                'offset': offset
            }
        )
        response.raise_for_status()  # Проверка на HTTP ошибки
        data = response.json().get('response', {}).get('items', [])
        all_posts.extend(data)
    except requests.RequestException as e:
        print(f"Ошибка выполнения запроса: {e}")
    except KeyError:
        print("Ошибка обработки данных из ответа")

    return all_posts

def file_writer(all_posts, domain, file_path):
    try:
        with open(file_path, 'a', newline='', encoding='utf-8') as file:
            a_pen = csv.writer(file)
            for post in all_posts:
                try:
                    post_id = post.get('id', 'N/A')
                    post_url = f"https://vk.com/wall{post['owner_id']}_{post_id}"
                    domain_url = f"https://vk.com/{domain}"
                    post_text = post.get('text', '')
                    a_pen.writerow((domain_url, post_url, post_text))
                except Exception as e:
                    print(f"Ошибка обработки поста: {e} - {post}")
                    continue 
    except IOError as e:
        print(f"Ошибка записи в файл: {e}")

def main():
    domains = ['chelnynews_ru', 'gazeta_rt', 'vestitatarstan', 'public141912684']  # список доменов для парсинга
    file_path = r"C:\Users\ilyal\OneDrive\Рабочий стол\hackaton\files\posts_new.csv"
    
    with open(file_path, 'w', newline='', encoding='utf-8') as file:
        a_pen = csv.writer(file)
        a_pen.writerow(('public_url', 'post_url', 'post_text'))

    total_posts = 0

    for domain in domains:
        offset = 0
        while True:
            posts = take_1000_posts(domain, offset)
            if not posts:
                print(f"Нет данных для домена: {domain}")
                break 
            file_writer(posts, domain, file_path)
            total_posts += len(posts)
            if total_posts >= 150000:
                print("Достигнуто максимально допустимое количество постов")
                break
            offset += len(posts)
            if len(posts) < 100:
                break 
        offset = 0 
        if total_posts >= 200000:
            break

    print(f"Парсинг завершен. Всего собрано постов: {total_posts}")

if __name__ == "__main__":
    main()

Достигнуто максимально допустимое количество постов
Достигнуто максимально допустимое количество постов
Достигнуто максимально допустимое количество постов
Достигнуто максимально допустимое количество постов
Парсинг завершен. Всего собрано постов: 150300
CPU times: total: 7min 7s
Wall time: 32min 13s


In [21]:
import pandas as pd 
df = pd.read_csv(r"C:\Users\ilyal\OneDrive\Рабочий стол\hackaton\files\posts_new.csv")

In [22]:
df

,public_url,post_url,post_text
0,https://vk.com/chelnynews_ru,https://vk.com/wall-54537775_2249270,В Альметьевске начнутся съемки российского сер...
1,https://vk.com/chelnynews_ru,https://vk.com/wall-54537775_2249266,Корпорат с Бастой и чиновниками устроила себе ...
2,https://vk.com/chelnynews_ru,https://vk.com/wall-54537775_2249265,Предлагаю вам послушать простую и честную песн...
3,https://vk.com/chelnynews_ru,https://vk.com/wall-54537775_2249263,29 августа Фестиваль современного искусства «Я...
4,https://vk.com/chelnynews_ru,https://vk.com/wall-54537775_2249261,🔥 Выиграй Яндекс Станцию Лайт в фотоконкурсе #...
...,...,...,...
150295,https://vk.com/public141912684,https://vk.com/wall-141912684_171854,«Пятый вагон пополам вообще сломан!»\n\nОчевид...
150296,https://vk.com/public141912684,https://vk.com/wall-141912684_171834,️ Новые кадра с места происшествия\n\nНа месте...
150297,https://vk.com/public141912684,https://vk.com/wall-141912684_171741,Наводнение в соседней Башкирии\n\nВсего подтоп...
150298,https://vk.com/public141912684,https://vk.com/wall-141912684_171732,Воздушная гимнастка из Казани упала из-под куп...
